In [1]:
# deep_optimal_stopping.ipynb
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import cholesky
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import norm
import os
import random
from torch import nn
# 配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

In [2]:
class StoppingPolicy(torch.nn.Module):
    """
    3层神经网络,包含批量归一化和Xavier初始化
    """
    def __init__(self, input_dim, hidden_dim=40):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, 1),
            torch.nn.Sigmoid()
        )
        # Xavier初始化
        for layer in self.net:
            if isinstance(layer, torch.nn.Linear):
                torch.nn.init.xavier_normal_(layer.weight)

    def forward(self, x):
        return self.net(x)

In [3]:
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)  # 全局固定随机种子

In [4]:
class PathGenerator:
    """路径生成的抽象基类"""
    def __init__(self, d, steps, T):
        self.d = d          # 维度
        self.steps = steps  # 时间步数
        self.T = T         # 总时间

    def generate(self, n_paths):
        raise NotImplementedError

In [5]:
class BlackScholesGenerator(PathGenerator):
    """
    多维Black-Scholes路径生成器
    """
    def __init__(self, d, steps, T, r, sigma, rho, div):
        super().__init__(d, steps, T)
        self.r = r
        self.sigma = sigma
        self.rho = rho
        self.div = div
        self.dt = T / steps
        self.drift = (self.r - self.div - 0.5 * self.sigma**2) * self.dt  # 新增此行
        self._build_cov_matrix()
        
    def _build_cov_matrix(self):
        """构建相关系数矩阵"""
        # 构造单位时间协方差矩阵（不乘 dt）
        cov = np.eye(self.d) * self.sigma**2
        cov += np.ones((self.d, self.d)) * (self.rho * self.sigma**2)
        np.fill_diagonal(cov, self.sigma**2)  # 防止对角重复加

        # 然后 Cholesky 分解
        self.L = torch.tensor(cholesky(cov, lower=True), device=device, dtype=torch.float32)
        
    def generate(self, n_paths):
        """生成路径 如有GPU可加速 """
        paths = torch.zeros(n_paths, self.steps+1, self.d, device=device, dtype=torch.float32)
        paths[:, 0] = 100.0  # 初始价格
        
        noise = torch.randn(n_paths, self.steps, self.d, device=device)
        
        sqrt_dt = torch.tensor(np.sqrt(self.dt), device=device, dtype=torch.float32)
        with tqdm(total=self.steps, desc="生成Black-Scholes路径", leave=False) as pbar:
            for t in range(1, self.steps+1):
                increments = self.drift + (self.L @ noise[:, t-1].T).T* sqrt_dt
                paths[:, t] = paths[:, t-1] * torch.exp(increments)
                pbar.update(1)
                
        return paths


In [6]:
class DeepOptimalStoppingTrainer:
    """
    实现论文第2节的递归训练算法
    """
    def __init__(self, generator, hidden_dim=40):
        self.generator = generator
        self.policies = self._init_policies(hidden_dim)
        self.optimizers = [torch.optim.Adam(p.parameters(), lr=0.001) 
                          for p in self.policies]
        
    def _init_policies(self, hidden_dim):
        input_dim = self.generator.d + 1  # 状态+收益
        # 初始化 steps+1 个策略网络（对应时间步 0~steps）
        return [StoppingPolicy(input_dim, hidden_dim).to(device)
                for _ in range(self.generator.steps + 1)]  # 修改为 steps+1
    
    def train(self, n_paths=8192, epochs=3000, batch_size=2048):
        """执行反向递归训练"""
        paths = self.generator.generate(n_paths)
        # 从到期日向前递归训练
        for step in tqdm(reversed(range(self.generator.steps)), desc="Training steps"):
            # 动态计算当前时间步的收益
            current_payoff = self._compute_payoffs(paths, start_step=step)[:, step]
            X, y = self._prepare_training_data(paths, current_payoff, step)
            self._train_single_step(step, X, y, epochs, batch_size)
    
    def _compute_payoffs(self, paths):
        """计算各时间步的即时收益（需子类实现）"""
        raise NotImplementedError
    
    def _prepare_training_data(self, paths, current_payoff, step):
        """准备训练数据（含嵌套蒙特卡洛）"""
        current_state = paths[:, step]
        X = torch.cat([current_state, current_payoff.unsqueeze(1)], dim=1)
        
        # 嵌套蒙特卡洛计算继续价值（传递起始时间步）
        continuation = self._nested_mc(paths, step)
        y = (current_payoff >= continuation).float()
        return X, y

    def _nested_mc(self, paths, step, J=13000):
        n_paths = paths.size(0)
        continuations = torch.zeros(n_paths, device=device)
        
        for i in range(n_paths):
            # 确保 current_state 是二维 [1, d]
            current_state = paths[i, step].unsqueeze(0)  # 添加批次维度
            new_paths = self._generate_continuations(current_state, step, J)
            
            # 应用策略网络生成停止时间
            ex_times = self._apply_policies(new_paths.squeeze(0), start_step=step)
            continuations[i] = self._compute_continuation_payoffs(
                new_paths.squeeze(0), ex_times, current_step=step
            ).mean()
        
        return continuations
    
    def _generate_continuations(self, current_state, step, J):
        """从当前状态生成继续路径（需子类实现）"""
        raise NotImplementedError
    
    def _apply_policies(self, paths, start_step):#以这个为准！！!
        """应用训练好的策略网络 论文公式5 """
        exercise_times = torch.full((paths.size(0),), self.generator.steps, device=device)
        for t in range(start_step + 1, paths.size(1)):
            states = paths[:, t]
            #print("是这里吗")
            # 动态计算当前时间步的收益，起始时间步为start_step
            payoffs = self._compute_payoffs(
                paths[:, :t+1],  # 截取到当前时间步的路径
                start_step=start_step
            )[:, t - start_step]  # 局部时间步索引
            
            inputs = torch.cat([states, payoffs.unsqueeze(1)], dim=1)
            stop_probs = self.policies[t](inputs).squeeze()
            stop_decisions = (stop_probs >= 0.5).float()
            
            mask = (exercise_times == self.generator.steps) & (stop_decisions == 1)
            exercise_times[mask] = t
        
        return exercise_times

    def _train_single_step(self, step, X, y, epochs, batch_size):
        """训练单个时间步的策略网络"""
        dataset = torch.utils.data.TensorDataset(X, y)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        for epoch in tqdm(range(epochs), desc=f"Training step {step}", leave=False):
            for batch_X, batch_y in loader:
                self.optimizers[step].zero_grad()
                pred = self.policies[step](batch_X).squeeze()
                loss = torch.nn.BCELoss()(pred, batch_y)
                loss.backward()
                self.optimizers[step].step()
    
    def compute_lower_bound(self, n_paths=4096000, conf_level=0.95):
        """计算下界及置信区间 论文3.1节 """
        # 生成独立路径集
        test_paths = self.generator.generate(n_paths)
        
        # 应用训练好的策略确定停止时间
        exercise_times = self._apply_policies(test_paths, 0)
        
        # 收集收益
        payoffs = torch.zeros(n_paths, device=device)
        for i in range(n_paths):
            t = int(exercise_times[i].item())
            payoffs[i] = self._compute_payoffs(test_paths[i:i+1])[0, t]
        
        # 计算统计量
        mean = payoffs.mean().item()
        std = payoffs.std(unbiased=True).item()
        z = 1.96 if conf_level == 0.95 else norm.ppf((1 + conf_level)/2)
        ci_half = z * std / np.sqrt(n_paths)
        
        return {
            'estimate': mean,
            'lower': mean - ci_half,
            'upper': mean + ci_half,
            'std': std
        }
    
    def compute_monte_carlo_price(self, test_paths_number=5000000):
        # 生成测试路径
        X= self.generator.generate(test_paths_number) # [test_paths_number, steps+1, d]# 计算每个路径在所有时间步的 payoff(贴现到时间0)
        g_values = self._compute_payoffs(X) # [test paths number, steps+1]
        # 利用训练好的策略确定每条路径的行权时刻
        # 这里采用_apply_policies，从时间0开始对整个路径应用策略网络
        exercise_times = self._apply_policies(X,0) # [test paths number
        # 对于每条路径，选取对应的 payoff
        exercise_times_np = exercise_times.cpu().numpy().astype(int)
        g_values_np = g_values.cpu().numpy()
        price_estimate = np.mean([g_values_np[i, t] for i,t in enumerate(exercise_times_np)])
        print(f"Monte Carlo Bermudan price: {price_estimate:.4f}")
        return price_estimate
    
    def compute_upper_bound(self, n_paths=100000, J=10000, conf_level=0.95):
        """基于给定公式计算上界"""
        # 生成主路径 [n_paths, steps+1, d]
        primary_paths = self.generator.generate(n_paths)
        payoffs = self._compute_payoffs(primary_paths, start_step=0)
        steps = self.generator.steps

        # 初始化鞅过程矩阵 [n_paths, steps+1]
        M = torch.zeros(n_paths, steps + 1, device=device)
        
        # 初始化延续价值矩阵 [n_paths, steps+1]
        C = torch.zeros(n_paths, steps + 1, device=device)
        
        # ========== 计算延续价值 C_n^k ==========
        for n in tqdm(range(steps), desc="预计算延续价值"):
            # 生成延续路径 [n_paths, J, steps_remaining, d]
            cont_paths = self._generate_continuations(primary_paths[:, n], n, J)
            
            # 应用策略网络生成停止时间 [n_paths, J]
            ex_times = self._apply_policies(cont_paths.view(-1, *cont_paths.shape[2:]), start_step=n)
            ex_times = ex_times.view(n_paths, J)
            
            # 计算每条路径的收益 [n_paths, J]
            cont_values = torch.zeros(n_paths, J, device=device)
            for i in range(n_paths):
                cont_values[i] = self._compute_continuation_payoffs(
                    cont_paths[i], ex_times[i], current_step=n
                )
            
            # 保存延续价值 C_n^k = E[g(τ_{n+1}) | F_n]
            C[:, n] = cont_values.mean(dim=1)
        
        # ========== 计算鞅增量 ΔM_n^k ==========
        for n in tqdm(range(1, steps + 1), desc="构建鞅过程"):
            # 获取当前状态 z_n^k [n_paths, d]
            states = primary_paths[:, n]
            
            # 计算策略函数值 f^{θ_n}(z_n^k) [n_paths]
            f_theta = self.policies[n](torch.cat([states, payoffs[:, n].unsqueeze(1)], dim=1)).squeeze()
            
            # 计算鞅增量 ΔM_n^k
            delta_M = (
                f_theta * payoffs[:, n] + 
                (1 - f_theta) * C[:, n] - 
                C[:, n-1]
            )
            
            # 累积鞅过程
            M[:, n] = M[:, n-1] + delta_M
        
        # ========== 计算上界估计 ==========
        # 计算 max(g(n,X_n) - M_n^k)
        adjusted_payoffs, _ = torch.max(payoffs - M, dim=1)
        
        # 统计结果
        mean_est = adjusted_payoffs.mean().item()
        std_est = adjusted_payoffs.std(unbiased=True).item()
        z = 1.96 if conf_level == 0.95 else norm.ppf((1 + conf_level) / 2)
        ci_half = z * std_est / np.sqrt(n_paths)
        
        return {
            'estimate': mean_est,
            'lower': mean_est - ci_half,
            'upper': mean_est + ci_half,
            'std': std_est
        }

    
    '''
    def compute_upper_bound(self, n_paths=1024, J=16384, conf_level=0.95):
        """计算上界及置信区间（严格遵循对偶公式）"""
        # 生成独立主路径
        primary_paths = self.generator.generate(n_paths)
        payoffs = self._compute_payoffs(primary_paths, start_step=0)
        
        # 初始化鞅过程 M 和条件期望缓存
        M = torch.zeros(n_paths, self.generator.steps + 1, device=device)
        E_H_prev = torch.zeros(n_paths, device=device)
        
        for t in tqdm(range(self.generator.steps + 1), desc="Building Martingale"):
            if t == 0:
                H_0 = self.compute_lower_bound(n_paths=n_paths)['estimate']
                M[:, 0] = 0
                E_H_prev[:] = H_0
                continue
            
            # 关键修正：计算正确的时间步数
            steps_remaining = self.generator.steps - (t-1)  # 剩余时间间隔数
            time_points = steps_remaining + 1  # 时间点数 = 间隔数 + 1
            
            # 初始化继续路径张量（修正第三维为time_points）
            cont_paths = torch.zeros(
                n_paths, 
                J, 
                time_points,  # 正确的时间点数
                self.generator.d, 
                device=device
            )
            
            # 生成继续路径并计算条件期望
            cont_values = torch.zeros(n_paths, J, device=device)
            for i in range(n_paths):
                current_state = primary_paths[i, t-1]
                cont_paths[i] = self._generate_continuations(current_state, t-1, J)
                ex_times = self._apply_policies(cont_paths[i], start_step=t-1)
                cont_values[i] = self._compute_continuation_payoffs(
                    cont_paths[i], 
                    ex_times, 
                    current_step=t-1
                )
            
            # 计算条件期望 E[H_t | F_{t-1}] = mean(cont_values)
            E_H_t = cont_values.mean(dim=1)  # 形状 (n_paths,)
            
            # 计算当前H_{t-1}（即主路径在t-1时刻的收益或继续价值）
            H_t_minus_1 = torch.maximum(payoffs[:, t-1], E_H_prev)
            
            avg_cont = E_H_prev.mean().item()
            avg_payoff = H_t_minus_1.mean().item()
            print(f"时间步 {t}: 平均主路径 payoff = {avg_payoff:.4f}, 平均 continuation value = {avg_cont:.4f}")

            # 更新鞅增量：M_t - M_{t-1} = H_t - E[H_t | F_{t-1}]
            delta_M = H_t_minus_1 - E_H_prev
            M[:, t] = M[:, t-1] + delta_M
            
            # 更新前一时间步的条件期望
            E_H_prev = E_H_t
        
        # 计算调整后的收益：max(g(n,X_n) - M_n)
        adjusted_payoffs = torch.zeros(n_paths, device=device)
        for i in range(n_paths):
            path_payoffs = payoffs[i]  # 主路径各时间步的收益
            adjusted_payoffs[i] = torch.max(path_payoffs - M[i, :len(path_payoffs)])
        
        # 计算统计量
        mean = adjusted_payoffs.mean().item()
        std = adjusted_payoffs.std(unbiased=True).item()
        z = 1.96 if conf_level == 0.95 else norm.ppf((1 + conf_level)/2)
        ci_half = z * std / np.sqrt(n_paths)
        
        return {
            'estimate': mean,
            'lower': mean - ci_half,
            'upper': mean + ci_half,
            'std': std
        }
    '''
    '''
    def compute_upper_bound(self, n_paths=1024, J=16384, conf_level=0.95):
        """计算上界及置信区间 论文3.2节双重方法"""
        # 生成主路径，确保生成的路径形状为 [n_paths, steps+1, d]
        primary_paths = self.generator.generate(n_paths)
        payoffs = self._compute_payoffs(primary_paths)  # shape: [n_paths, steps+1]
        
        # 初始化Martingale, M[:,0] = 0，M的形状 [n_paths, steps+1]
        M = torch.zeros(n_paths, self.generator.steps + 1, device=device)
        
        # 对于每个时间步 t 从 0 到 steps-1 进行循环
        for t in range(self.generator.steps):
            steps_remaining = self.generator.steps - t  # 剩余的时间步数
            # 构造 continuation paths：形状 [n_paths, J, steps_remaining+1, d]
            cont_paths = torch.zeros(
                n_paths,
                J,
                steps_remaining + 1,  # 注意：这里+1使得包含起始时刻
                self.generator.d,
                device=device
            )
            
            # 为每条主路径生成对应的 continuation paths
            for i in range(n_paths):
                # _generate_continuations 接受当前状态 primary_paths[i,t] 和当前时间 t，
                # 返回形状 [J, steps_remaining+1, d] 的 continuation paths
                cont_paths[i] = self._generate_continuations(primary_paths[i, t], t, J)
            
            # 计算每条主路径在 t 时刻的继续价值 cont_value
            cont_values = torch.zeros(n_paths, device=device)
            for i in range(n_paths):
                # 对于每个主路径，使用生成的 continuation paths 和当前时间 t，
                # 计算停止策略下的停止时刻
                ex_times = self._apply_policies(cont_paths[i], t)
                # 计算 continuation payoff（需要确保 _compute_continuation_payoffs 正确使用 current_step=t 进行折现）
                # 返回的是一个 [J] 维的张量，再取均值
                cont_payoffs = self._compute_continuation_payoffs(cont_paths[i], ex_times, current_step=t)
                cont_values[i] = cont_payoffs.mean()
            
            # Debug
            current_payoff = payoffs[:, t]  # shape: [n_paths]
            avg_cont = cont_values.mean().item()
            avg_payoff = current_payoff.mean().item()
            print(f"时间步 {t}: 平均主路径 payoff = {avg_payoff:.4f}, 平均 continuation value = {avg_cont:.4f}")
            
            delta_M = (current_payoff >= cont_values).float() * (current_payoff - cont_values)
            M[:, t + 1] = M[:, t] + delta_M
        
        # 调整后的收益：payoff - martingale，取路径上的最大值
        adjusted = payoffs - M
        max_values, _ = torch.max(adjusted, dim=1)
        
        # 计算统计量
        mean = max_values.mean().item()
        std = max_values.std(unbiased=True).item()
        z = 1.96 if conf_level == 0.95 else norm.ppf((1 + conf_level) / 2)
        ci_half = z * std / np.sqrt(n_paths)
        
        print(f"上界估计: {mean:.4f}, 95% CI: [{mean - ci_half:.4f}, {mean + ci_half:.4f}]")
        return {
            'estimate': mean,
            'lower': mean - ci_half,
            'upper': mean + ci_half,
            'std': std
        }
        '''

In [7]:
# ====================== 策略网络 ======================
class StoppingPolicy(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.net(x)

# ====================== 训练器类 ======================
class newDeepOptimalStoppingTrainer:
    def __init__(self, generator, hidden_dim=40):
        self.generator = generator
        self.policies = self._init_policies(hidden_dim)
        self.optimizers = [torch.optim.Adam(p.parameters(), lr=0.001) for p in self.policies]
        
    def _init_policies(self, hidden_dim):
        input_dim = self.generator.d + 1
        return [StoppingPolicy(input_dim, hidden_dim).to(device) for _ in range(self.generator.steps + 1)]
    
    def train(self, n_paths=8192, epochs=3000, batch_size=2048):
        """确定性训练过程"""
        paths = self.generator.generate(n_paths)
        payoffs = self._compute_payoffs(paths)
        
        for step in tqdm(reversed(range(self.generator.steps)), desc="Training"):
            current_payoff = payoffs[:, step]
            continuation = self._compute_continuation(paths, payoffs, step)
            
            X = torch.cat([paths[:, step], current_payoff.unsqueeze(1)], dim=1)
            y = (current_payoff >= continuation).float()
            
            self._train_single_step(step, X, y, epochs, batch_size)
    
    def _compute_continuation(self, paths, payoffs, step):
        if step + 1 > self.generator.steps:
            return torch.zeros(paths.size(0), device=device)
        
        sub_paths = paths[:, step+1:]
        exercise_times = self._apply_policies(sub_paths, start_step=step+1)
        exercise_times = torch.clamp(exercise_times, max=self.generator.steps)
        return payoffs[torch.arange(len(exercise_times)), exercise_times.long()]
    
    def _apply_policies(self, paths, start_step):
        """确定性策略应用"""
        batch_size, total_steps, _ = paths.size()
        exercise_times = torch.full((batch_size,), self.generator.steps, device=device)
        
        for local_t in range(total_steps):
            global_t = start_step + local_t + 1  # 确保行权时间≥start_step+1
            if global_t > self.generator.steps:
                break
            
            states = paths[:, local_t]
            partial_paths = paths[:, :local_t+1]
            current_payoff = self._compute_payoffs(partial_paths, start_step=start_step)[:, local_t]
            
            inputs = torch.cat([states, current_payoff.unsqueeze(1)], dim=1)
            with torch.no_grad():  # 关闭梯度计算
                stop_probs = self.policies[global_t](inputs).squeeze()
            
            stop_decisions = (stop_probs >= 0.5).float()
            mask = (exercise_times == self.generator.steps) & (stop_decisions == 1)
            exercise_times[mask] = global_t
        
        return exercise_times

    def _train_single_step(self, step, X, y, epochs, batch_size):
        """确定性数据加载"""
        if X.size(0) < 2:
            raise ValueError(f"训练数据不足: {X.size(0)} 样本")
        
        # 固定数据加载的随机性
        g = torch.Generator().manual_seed(42)
        dataset = TensorDataset(X, y)
        loader = DataLoader(dataset, batch_size=max(2, batch_size), shuffle=True, generator=g)
        
        for _ in range(epochs):
            for batch_X, batch_y in loader:
                self.optimizers[step].zero_grad()
                pred = self.policies[step](batch_X).squeeze()
                loss = nn.BCELoss()(pred, batch_y)
                loss.backward()
                self.optimizers[step].step()
    
    def _compute_payoffs(self, paths, start_step=0):
        """Bermudan Max-Call收益计算"""
        max_values, _ = torch.max(paths, dim=2)
        steps_remaining = paths.size(1) - 1
        
        time_points = torch.linspace(
            start_step * self.generator.dt,
            self.generator.dt * (start_step + steps_remaining),
            steps_remaining + 1,
            device=device
        )
        discounts = torch.exp(-self.generator.r * time_points)
        return (max_values - 100).clamp(min=0) * discounts
    
    def compute_lower_bound(self, n_paths=4096000, conf_level=0.95):
        """确定性下界计算"""
        test_paths = self.generator.generate(n_paths)
        exercise_times = self._apply_policies(test_paths, 0)
        
        payoffs = torch.zeros(n_paths, device=device)
        for i in range(n_paths):
            payoffs[i] = self._compute_payoffs(test_paths[i:i+1])[0, int(exercise_times[i])]
        
        mean = payoffs.mean().item()
        std = payoffs.std(unbiased=True).item()
        z = norm.ppf((1 + conf_level)/2)
        return {
            'estimate': mean,
            'lower': mean - z * std / np.sqrt(n_paths),
            'upper': mean + z * std / np.sqrt(n_paths)
        }
    
    def compute_upper_bound(self, n_paths=1024, J=16384, conf_level=0.95):
        """确定性上界计算"""
        primary_paths = self.generator.generate(n_paths)
        payoffs = self._compute_payoffs(primary_paths)
        steps = self.generator.steps
        
        # 计算平均行权时间
        exercise_times = torch.full((n_paths,), steps, device=device)
        for t in range(steps):
            states = primary_paths[:, t]
            current_payoff = payoffs[:, t]
            inputs = torch.cat([states, current_payoff.unsqueeze(1)], dim=1)
            with torch.no_grad():
                stop_probs = self.policies[t](inputs).squeeze()
            stop_decisions = (stop_probs >= 0.5).float()
            mask = (exercise_times == steps) & (stop_decisions == 1)
            exercise_times[mask] = t
        avg_time = exercise_times.float().mean().item()
        
        # 计算鞅过程
        M = torch.zeros(n_paths, steps + 1, device=device)
        E_H_prev = payoffs[:, 0].clone()
        
        for t in tqdm(range(1, steps + 1), desc="Building Martingale"):
            steps_remaining = steps - (t - 1)
            cont_paths = torch.zeros(n_paths, J, steps_remaining + 1, self.generator.d, device=device)
            
            for i in range(n_paths):
                current_state = primary_paths[i, t-1]
                cont_paths[i] = self.generator.generate(J)[:, :steps_remaining + 1]  # 使用固定种子生成
                ex_times = self._apply_policies(cont_paths[i], start_step=t-1)
                cont_values = self._compute_payoffs(cont_paths[i], start_step=t-1)[
                    torch.arange(J), ex_times.long()
                ]
                E_H_prev[i] = cont_values.mean()
            
            delta_M = torch.clamp(payoffs[:, t-1] - E_H_prev, min=0)
            M[:, t] = M[:, t-1] + delta_M
            E_H_prev = cont_values.mean(dim=0)
        
        adjusted = payoffs - M[:, :payoffs.size(1)]
        max_values = adjusted.max(dim=1)[0]
        
        mean = max_values.mean().item()
        std = max_values.std(unbiased=True).item()
        z = norm.ppf((1 + conf_level)/2)
        return {
            'estimate': mean,
            'lower': mean - z * std / np.sqrt(n_paths),
            'upper': mean + z * std / np.sqrt(n_paths),
            'avg_exercise_time': avg_time
        }


In [7]:
class BermudanMaxCallTrainer(DeepOptimalStoppingTrainer):
    """实现论文第4.1节的Bermudan Max-Call实验"""
    def _compute_payoffs(self, paths, start_step=0):
        # paths shape: (n_paths, L, d)，其中 L = 实际时间点数
        max_values, _ = torch.max(paths, dim=2)  # (n_paths, L)
        L = paths.size(1)  # 实际时间点数
        # 用离散时间步构造时间向量：start_time, start_time+dt, …, start_time+(L-1)*dt
        time_points = start_step * self.generator.dt + torch.arange(L, device=device, dtype=torch.float32) * self.generator.dt
        discounts = torch.exp(-self.generator.r * time_points)
        return (max_values - 100).clamp(min=0) * discounts  # K=100

    '''
    def _compute_payoffs(self, paths, start_step=0):
        max_values, _ = torch.max(paths, dim=2)  # (n_paths, steps+1)
        steps_remaining = paths.size(1) - 1  # 路径的局部时间步数
        
        start_time = start_step * self.generator.dt
        time_points = torch.linspace(start_time, self.generator.T, steps_remaining + 1, device=device)
        discounts = torch.exp(-self.generator.r * time_points)
        return (max_values - 100).clamp(min=0) * discounts  # K=100
'''
    def _generate_continuations(self, current_state, step, J):
        """生成延续路径（自动处理一维输入）"""
        # 检查输入维度，确保为二维 [batch_size, d]
        if current_state.dim() == 1:
            current_state = current_state.unsqueeze(0)  # 转换为 [1, d]
        batch_size, d = current_state.shape
        
        steps_remaining = self.generator.steps - step
        dt = self.generator.dt
        sqrt_dt = torch.sqrt(torch.tensor(dt, device=device))

        # 初始化路径张量 [batch_size, J, steps_remaining+1, d]
        new_paths = torch.zeros(batch_size, J, steps_remaining + 1, d, 
                            device=device, dtype=torch.float32)
        
        # 广播初始状态到所有J条路径 [batch_size, J, d]
        new_paths[:, :, 0] = current_state.unsqueeze(1).expand(-1, J, -1)
        
        # 预生成随机数 [batch_size, J, steps_remaining, d]
        Z = torch.randn(batch_size, J, steps_remaining, d, device=device)
        
        # 向量化路径生成
        for t in range(1, steps_remaining + 1):
            # 计算相关噪声 [batch_size, J, d]
            noise = Z[:, :, t-1] @ self.generator.L.T  # 矩阵乘法引入相关性
            
            # 计算增量 [batch_size, J, d]
            increments = self.generator.drift + noise * sqrt_dt
            
            # 更新路径 [batch_size, J, d]
            new_paths[:, :, t] = new_paths[:, :, t-1] * torch.exp(increments)
        
        return new_paths
    
    def _compute_continuation_payoffs(self, new_paths, exercise_times, current_step):
        # new_paths shape: (batch_size, L, d)，其中 L = steps_remaining + 1
        batch_size, L, _ = new_paths.shape
        time_points = current_step * self.generator.dt + torch.arange(L, device=device, dtype=torch.float32) * self.generator.dt
        # 排除当前时刻（index 0）
        discounts = torch.exp(-self.generator.r * time_points)[1:]
        
        max_values, _ = torch.max(new_paths, dim=2)  # (batch_size, L)
        payoffs = (max_values[:, 1:] - 100).clamp(min=0) * discounts  # 从 current_step+1 开始
        
        selected_payoffs = torch.zeros(batch_size, device=device)
        for i in range(batch_size):
            absolute_t = int(exercise_times[i].item())
            if absolute_t == self.generator.steps:  # 到期时执行
                selected_payoffs[i] = payoffs[i, -1]
            else:
                relative_t = absolute_t - (current_step + 1)
                if relative_t < 0 or relative_t >= (L - 1):
                    raise ValueError(f"Invalid exercise time {absolute_t} at step {current_step}")
                selected_payoffs[i] = payoffs[i, relative_t]
        
        return selected_payoffs
    
    '''
    def _compute_continuation_payoffs(self, new_paths, exercise_times, current_step):
        batch_size, steps_remaining_plus1, _ = new_paths.shape
        steps_remaining = steps_remaining_plus1 - 1  # 剩余时间间隔数
        
        # 时间点从 current_step + 1 到 T
        start_time = (current_step + 1) * self.generator.dt
        time_points = torch.linspace(start_time, self.generator.T, steps_remaining_plus1, device=device)
        discounts = torch.exp(-self.generator.r * (time_points - current_step * self.generator.dt))[1:]  # 排除当前时间步
        
        max_values, _ = torch.max(new_paths, dim=2)  # 形状 (batch_size, steps_remaining_plus1)
        payoffs = (max_values[:, 1:] - 100).clamp(min=0) * discounts  # 从下一步开始
        
        selected_payoffs = torch.zeros(batch_size, device=device)
        for i in range(batch_size):
            absolute_t = int(exercise_times[i].item())
            # 确保行权时间在合法范围内 [current_step + 1, steps]
            if absolute_t < (current_step + 1) or absolute_t > self.generator.steps:
                selected_payoffs[i] = payoffs[i, -1]  # 若越界，强制在到期日行权
            else:
                relative_t = absolute_t - (current_step + 1)
                if relative_t < 0 or relative_t >= steps_remaining:
                    selected_payoffs[i] = payoffs[i, -1]
                else:
                    selected_payoffs[i] = payoffs[i, relative_t]
        
        return selected_payoffs
    
    def _compute_continuation_payoffs(self, new_paths, exercise_times, current_step):
        batch_size, steps_remaining_plus1, _ = new_paths.shape
        steps_remaining = steps_remaining_plus1 - 1  # 时间间隔数
        
        # 时间点从current_step到current_step + steps_remaining（含两端）
        time_points = torch.linspace(
            current_step * self.generator.dt,
            self.generator.T,
            steps_remaining_plus1,
            device=device
        )
        discounts = torch.exp(-self.generator.r * time_points)[1:]  # 排除current_step
        
        max_values, _ = torch.max(new_paths, dim=2)  # (batch_size, steps_remaining+1)
        payoffs = (max_values[:, 1:] - 100).clamp(min=0) * discounts  # 从current_step+1开始
        
        selected_payoffs = torch.zeros(batch_size, device=device)
        for i in range(batch_size):
            absolute_t = int(exercise_times[i].item())
            if absolute_t == self.generator.steps:  # 到期日执行
                selected_payoffs[i] = payoffs[i, -1]
            else:
                # 转换为继续路径的局部索引（从current_step+1开始）
                relative_t = absolute_t - (current_step + 1)
                if relative_t < 0 or relative_t >= steps_remaining:
                    raise ValueError(f"Invalid exercise time {absolute_t} at step {current_step}")
                selected_payoffs[i] = payoffs[i, relative_t]
        
        return selected_payoffs
    '''

In [9]:
class MBRCGenerator(BlackScholesGenerator):
    """带股息支付和障碍监测的路径生成器"""
    def __init__(self, d, steps, T, r, sigma, rho, div_dates, div_rates, barriers):
        super().__init__(d, steps, T, r, sigma, rho, div=0)
        self.div_dates = div_dates  # 股息支付时间索引列表
        self.div_rates = div_rates  # 各资产股息率
        self.barriers = barriers    # 障碍水平（百分比）

    def generate(self, n_paths):
        paths = super().generate(n_paths)
        # 检查股息支付时间步是否在路径范围内
        for t in self.div_dates:
            if t >= self.steps:
                raise ValueError(f"Dividend date {t} exceeds path steps {self.steps}")
            paths[:, t+1:] *= (1 - torch.tensor(self.div_rates, device=device))
        return paths

    def track_barriers(self, paths):
        """监测障碍事件（适配局部路径的时间窗口）"""
        barrier_hit = torch.zeros_like(paths[:, :, 0], dtype=torch.bool)
        for t_local in range(paths.size(1)):  # 局部时间步
            current_min = torch.min(paths[:, t_local, :], dim=1).values
            barrier_hit[:, t_local] = (current_min <= self.barriers * 100)
        return barrier_hit

In [10]:
class CallableMBRCTrainer(DeepOptimalStoppingTrainer):
    """MBRC训练器 最小化发行人成本"""
    def __init__(self, generator, coupon_rate, nominal=100):
        super().__init__(generator)
        self.coupon = coupon_rate * generator.T / generator.steps
        self.nominal = nominal

    def _compute_payoffs(self, paths, start_step=0):
        barrier_indicator = self.generator.track_barriers(paths)
        n_paths, total_time_steps, _ = paths.shape
        
        start_time = start_step * self.generator.dt
        time_points = torch.linspace(
            start_time,
            self.generator.T,
            total_time_steps,
            device=device
        )
        discounts = torch.exp(-self.generator.r * time_points)
        
        payoffs = torch.zeros(n_paths, total_time_steps, device=device)
        
        for t in range(total_time_steps):
            global_t = start_step + t
            global_t = min(global_t, self.generator.steps)  # 确保不越界
            
            
            coupon_part = self.coupon * (global_t + 1)
            
            if global_t == self.generator.steps:
                final_prices = paths[:, t, :]
                min_price = torch.min(final_prices, dim=1).values
                # 使用 torch.minimum 替换 torch.min
                payoff = torch.where(
                    barrier_indicator[:, global_t],
                    torch.minimum(min_price, torch.tensor(self.nominal, device=device)),
                    self.nominal
                )
            else:
                payoff = self.nominal
            
            payoffs[:, t] = (coupon_part + payoff) * discounts[t]
        
        return payoffs
    
    def _generate_continuations(self, current_state, step, J):
        """生成考虑股息的继续路径"""
        new_paths = torch.zeros(J, self.generator.steps-step, self.generator.d, device=device)
        new_paths[:, 0] = current_state
        
        for t in range(1, self.generator.steps-step):
            Z = torch.randn(J, self.generator.d, device=device)
            increments = self.generator.drift + (self.generator.L @ Z.T).T * np.sqrt(self.generator.dt)
            new_paths[:, t] = new_paths[:, t-1] * torch.exp(increments)
            
            # 应用股息支付
            if (step + t) in self.generator.div_dates:
                new_paths[:, t:] *= (1 - torch.tensor(self.generator.div_rates, device=device))
        
        return new_paths

    def _compute_continuation_payoffs(self, new_paths, exercise_times, current_step):
        barrier_indicator = self.generator.track_barriers(new_paths)
        batch_size, steps_remaining, _ = new_paths.shape
        
        start_time = current_step * self.generator.dt
        time_points = torch.linspace(
            start_time,
            self.generator.T,
            steps_remaining + 1,
            device=device
        )
        discounts = torch.exp(-self.generator.r * time_points)[1:]
        
        payoffs = torch.zeros(batch_size, device=device)
        for i in range(batch_size):
            t = int(exercise_times[i].item())
            relative_t = t - current_step - 1
            
            if relative_t < 0 or relative_t >= steps_remaining:
                raise ValueError(f"Invalid exercise time {t} at step {current_step}")
            
            # 全局时间步截断
            global_t = min(current_step + relative_t + 1, self.generator.steps)
            
            coupon_part = self.coupon * (global_t + 1)
            
            if global_t == self.generator.steps:
                final_prices = new_paths[i, relative_t, :]
                min_price = torch.min(final_prices)
                payoff = torch.where(
                    barrier_indicator[i, relative_t],  # 使用局部索引
                    torch.minimum(min_price, torch.tensor(self.nominal, device=device)),
                    self.nominal
                )
            else:
                payoff = self.nominal
            
            discounted_payoff = (coupon_part + payoff) * discounts[relative_t]
            payoffs[i] = discounted_payoff
        
        return payoffs

In [11]:
class FBMGenerator(PathGenerator):
    """分数布朗运动生成器（带状态嵌入）"""
    def __init__(self, H, steps, T):
        super().__init__(steps+1, steps, T)  # 状态维度=时间步数
        self.H = H
        self._build_cov_matrix()
    
    def _build_cov_matrix(self):
        t = np.linspace(0, self.T, self.steps+1)
        cov = 0.5 * (
            t[:, None]**(2 * self.H) 
            + t[None, :]**(2 * self.H) 
            - np.abs(t[:, None] - t[None, :])**(2 * self.H)
        )
        np.fill_diagonal(cov, cov.diagonal() + 1e-6)
        
        # 转换为PyTorch张量并保存
        self.cov_matrix = torch.tensor(
            cov.astype(np.float32),  # 先转换为float32的NumPy数组
            device=device,
            dtype=torch.float32       # 再转为PyTorch张量
        )
        
        # Cholesky分解
        self.L = torch.linalg.cholesky(self.cov_matrix)
        
    def generate(self, n_paths):
        paths = torch.zeros(n_paths, self.steps+1, device=device, dtype=torch.float32)
        for i in range(n_paths):
            Z = torch.randn(self.steps+1, device=device, dtype=torch.float32)
            paths[i] = self.L @ Z
        
        # 关键：生成三维嵌入路径，形状 (n_paths, steps+1, steps+1)
        embedded = torch.zeros(n_paths, self.steps+1, self.steps+1, device=device)
        for t in range(self.steps+1):
            embedded[:, t, :t+1] = paths[:, :t+1]  # 填充完整历史状态
        return embedded

class FBMTrainer(DeepOptimalStoppingTrainer):
    """分数布朗运动训练器"""
    def __init__(self, H, steps, T, hidden_dim=140):
        generator = FBMGenerator(H, steps, T)
        super().__init__(generator)
        input_dim = (steps + 1) + 1  # 输入维度 = 状态(steps+1) + 收益(1)
        self.policies = [
            StoppingPolicy(input_dim, hidden_dim).to(device)
            for _ in range(steps + 1)
        ]
        self.optimizers = [torch.optim.Adam(p.parameters(), lr=0.001) for p in self.policies]

    def _prepare_training_data(self, paths, current_payoff, step):
        # paths形状应为 (n_paths, steps+1, steps+1)
        current_state = paths[:, step, :]  # 提取当前时间步的所有历史特征，形状 (n_paths, steps+1)
        X = torch.cat([current_state, current_payoff.unsqueeze(1)], dim=1)  # 形状 (n_paths, steps+2)
        continuation = self._nested_mc(paths, step)
        y = (current_payoff >= continuation).float()
        return X, y

    def _apply_policies(self, paths, start_step):
        exercise_times = torch.full((paths.size(0),), self.generator.steps, device=device)
        for t in range(start_step + 1, self.generator.steps + 1):
            # 检查路径是否包含足够的时间步
            if t >= paths.size(1):
                break  # 防止越界
            
            states = paths[:, t, :]
            payoffs = self._compute_payoffs(paths[:, :t+1, :], start_step=start_step)[:, t - start_step]
            inputs = torch.cat([states, payoffs.unsqueeze(1)], dim=1)
            stop_probs = self.policies[t](inputs).squeeze()
            stop_decisions = (stop_probs >= 0.5).float()
            
            mask = (exercise_times == self.generator.steps) & (stop_decisions == 1)
            exercise_times[mask] = t
        return exercise_times

    def _compute_payoffs(self, paths, start_step=0):
        """从三维路径中提取FBM值"""
        # paths形状应为 (n_paths, steps_remaining+1, features)
        n_paths, steps_remaining_plus_1, features = paths.shape
        payoffs = torch.zeros(n_paths, steps_remaining_plus_1, device=device)
        for t in range(steps_remaining_plus_1):
            payoffs[:, t] = paths[:, t, t]  # 提取对角线特征（假设特征在最后一维）
        return payoffs

    def _generate_continuations(self, current_state, step, J):
        """生成条件路径（严格匹配步数）"""
        t = step  # 当前时间步（从0开始）
        total_steps = self.generator.steps
        
        # 需要生成的未来步数
        future_steps = total_steps - t  # 从 t+1 到 total_steps，共 future_steps 步
        
        # 获取协方差矩阵
        cov = self.generator.cov_matrix  # 形状 (total_steps+1, total_steps+1)
        
        # 分割协方差矩阵
        Sigma11 = cov[:t+1, :t+1]          # 历史路径协方差 (t+1, t+1)
        Sigma12 = cov[:t+1, t+1:t+1+future_steps]  # 历史与未来的协方差 (t+1, future_steps)
        Sigma22 = cov[t+1:t+1+future_steps, t+1:t+1+future_steps]  # 未来路径协方差 (future_steps, future_steps)
        
        # 计算条件协方差
        Sigma22_1 = Sigma22 - Sigma12.T @ torch.linalg.inv(Sigma11) @ Sigma12
        L22 = torch.linalg.cholesky(Sigma22_1)
        
        # 生成随机噪声
        Z = torch.randn(J, future_steps, device=device)
        
        # 计算均值项和随机项
        current_state_truncated = current_state[:t+1]  # 截取到当前时间步（包含t）
        mean_part = current_state_truncated @ torch.linalg.pinv(Sigma11) @ Sigma12  # 形状 (1, future_steps)
        random_part = (L22 @ Z.T).T  # 形状 (J, future_steps)
        
        # 合并均值和随机部分
        continuation_values = mean_part + random_part  # 形状 (J, future_steps)
        
        # 初始化三维路径张量 (J, total_steps+1, total_steps+1)
        cond_paths = torch.zeros(J, total_steps+1, total_steps+1, device=device)
        
        # 填充历史路径（第一个特征维度）
        cond_paths[:, :t+1, 0] = current_state_truncated.expand(J, -1)
        
        # 填充未来路径到对应特征位置（从 t+1 开始）
        cond_paths[:, t+1:t+1+future_steps, t+1:t+1+future_steps] = continuation_values.unsqueeze(-1)
        
        return cond_paths
    
    def _compute_continuation_payoffs(self, new_paths, exercise_times, current_step):
        """计算继续路径的收益 直接提取FBM值 """
        payoffs = torch.zeros(new_paths.size(0), device=device)
        for i in range(new_paths.size(0)):
            t = int(exercise_times[i].item())
            # 转换为继续路径的局部索引
            relative_t = t - current_step - 1
            if relative_t < 0 or relative_t >= new_paths.size(1):
                raise ValueError(f"Invalid exercise time {t} at step {current_step}")
            payoffs[i] = new_paths[i, relative_t, relative_t]  # 提取局部路径的FBM值
        return payoffs

    def compute_upper_bound(self, n_paths=1024, J=16384, conf_level=0.95):
        """计算上界（修正继续路径步数）"""
        # 生成主路径
        primary_paths = self.generator.generate(n_paths)
        payoffs = self._compute_payoffs(primary_paths)
        
        # 初始化Martingale过程
        M = torch.zeros(n_paths, self.generator.steps+1, device=device)
        
        # 计算Martingale增量
        for t in tqdm(range(self.generator.steps), desc="计算上界(Martingale)"):
            # 生成继续路径的步数应为 (steps - t)
            steps_remaining = self.generator.steps - t
            
            # 初始化继续路径容器，形状 (n_paths, J, steps_remaining, features)
            cont_paths = torch.zeros(
                n_paths, J, steps_remaining, self.generator.d,  # 修正步数为 steps_remaining
                device=device
            )
            
            # 生成继续路径
            for i in range(n_paths):
                # 生成从 t 开始的继续路径（步数为 steps_remaining）
                full_cont_path = self._generate_continuations(primary_paths[i, t], t, J)
                # 截取从 t+1 到 steps 的路径（共 steps_remaining 步）
                cont_paths[i] = full_cont_path[:, t+1:t+1+steps_remaining, :]
            
            # 计算继续价值
            cont_values = torch.zeros(n_paths, device=device)
            for i in range(n_paths):
                ex_times = self._apply_policies(cont_paths[i], t)
                cont_values[i] = self._compute_continuation_payoffs(cont_paths[i], ex_times, t).mean()
            
            # 更新Martingale
            current_payoff = payoffs[:, t]
            delta_M = (current_payoff >= cont_values).float() * (current_payoff - cont_values)
            M[:, t+1] = M[:, t] + delta_M
        
        # 计算调整后的收益
        adjusted = payoffs - M
        max_values, _ = torch.max(adjusted, dim=1)
        
        # 统计量和置信区间计算（保持不变）
        mean = max_values.mean().item()
        std = max_values.std(unbiased=True).item()
        z = 1.96 if conf_level == 0.95 else norm.ppf((1 + conf_level)/2)
        ci_half = z * std / np.sqrt(n_paths)
        
        return {
            'estimate': mean,
            'lower': mean - ci_half,
            'upper': mean + ci_half,
            'std': std
        }

In [12]:
class FBMTrainerWithCustomPrepare(DeepOptimalStoppingTrainer):
    """FBM专用训练器, 完全覆盖所有必要方法"""
    def __init__(self, H, steps, T, hidden_dim=140):
        generator = FBMGenerator(H, steps, T)
        super().__init__(generator)
        input_dim = (steps + 1) + 1  # 输入维度 = 状态(steps+1) + 收益(1)
        self.policies = [
            StoppingPolicy(input_dim, hidden_dim).to(device)
            for _ in range(steps + 1)
        ]
        self.optimizers = [torch.optim.Adam(p.parameters(), lr=0.001) for p in self.policies]

    # -------------------- 核心方法覆盖 --------------------
    def _prepare_training_data(self, paths, current_payoff, step):
        """FBM专用数据准备（保持其他模型不变）"""
        current_state = paths[:, step, :]  # 从三维嵌入路径提取状态
        X = torch.cat([current_state, current_payoff.unsqueeze(1)], dim=1)
        continuation = self._nested_mc(paths, step)
        y = (current_payoff >= continuation).float()
        return X, y

    def _compute_payoffs(self, paths, start_step=0):
        """从三维路径中提取FBM值"""
        n_paths, steps_remaining, _ = paths.shape
        payoffs = torch.zeros(n_paths, steps_remaining, device=device)
        for t in range(steps_remaining):
            payoffs[:, t] = paths[:, t, t]  # 关键：提取对角线值
        return payoffs

    def _generate_continuations(self, current_state, step, J):
        """生成条件路径（严格三维结构）"""
        t = step + 1
        cond_paths = torch.zeros(J, self.generator.steps+1, self.generator.steps+1, device=device)
        cond_paths[:, :t, :t] = current_state.expand(J, -1, -1)  # 复制历史路径
        
        cov = self.generator.cov_matrix
        Sigma11 = cov[:t, :t]
        Sigma12 = cov[:t, t:]
        Sigma22 = cov[t:, t:]
        Sigma22_1 = Sigma22 - Sigma12.T @ torch.linalg.inv(Sigma11) @ Sigma12
        L22 = torch.linalg.cholesky(Sigma22_1)
        
        Z = torch.randn(J, self.generator.steps+1 - t, device=device)
        cond_paths[:, t:, t:] = (current_state[:t] @ torch.linalg.pinv(Sigma11) @ Sigma12) + (L22 @ Z.T).T
        return cond_paths

    def _compute_continuation_payoffs(self, new_paths, exercise_times, current_step):
        """从继续路径中提取收益"""
        payoffs = torch.zeros(new_paths.size(0), device=device)
        for i in range(new_paths.size(0)):
            t = int(exercise_times[i].item())
            relative_t = t - current_step - 1
            if 0 <= relative_t < new_paths.size(1):
                payoffs[i] = new_paths[i, relative_t, relative_t]
            else:
                payoffs[i] = 0.0  # 无效时间步处理
        return payoffs

In [13]:
# 实验执行代码
def run_Bermudan_experiment():
    """Bermudan实验 对应论文表1/2 """
    params = {
        "d": 3,
        "steps": 9,
        "T": 3.0,
        "r": 0.05,
        "sigma": 0.2,
        "rho": 0.0,
        "div": 0.1
    }
    
    # 初始化生成器和训练器
    bs_gen = BlackScholesGenerator(**params)
    print("BlackScholesGenerator steps:", bs_gen.steps)
    trainer = BermudanMaxCallTrainer(bs_gen, hidden_dim=40+params["d"])
    
    # 训练（3000步，每批8192条路径）
    print("开始训练...")
    trainer.train(n_paths=4000, epochs=1500+params["d"],batch_size=2048)#8192,3000
    
    # 计算上下界
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)#4096000 2000000
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000) #1024，16384

    del trainer
    del bs_gen

    # 输出结果
    print(f"\nBermudan实验结果:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['lower']:.3f}, {ub_result['upper']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")

    del lb_result
    del ub_result

def run_mbrc_experiment():
    """Callable MBRC实验 对应论文表3 """
    params = {
        "d": 5,
        "steps": 252,       # 252，1年，每日监测
        "T": 1.0,
        "r": 0.00,
        "sigma": 0.2,
        "rho": 0.0,
        "div_dates": [126],  # 半年支付股息，126
        "div_rates": [0.05]*5,
        "barriers": 0.7      # 70%障碍
    }
    
    mbrc_gen = MBRCGenerator(**params)
    trainer = CallableMBRCTrainer(mbrc_gen, coupon_rate=0.07, nominal=100,hidden_dim=40+params["d"])
    
    print("Training Callable MBRC policies...")
    trainer.train(n_paths=8192, epochs=3000+params["d"])#8192,3000
    
    # 计算上下界
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=4096000)#4096000
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000)#1024，16384
    
    
    # 输出结果
    print(f"\nMBRC实验结果:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['upper']:.3f}, {ub_result['lower']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")


def run_fbm_experiment(H=0.7):
    """分数布朗运动实验 对应论文表4 """
    assert 0 < H < 1, "Hurst指数必须在 (0, 1) 之间"
    params = {
        "H": H,
        "steps": 100,#100
        "T": 1.0
    }
    
    fbm_gen = FBMGenerator(**params)
    trainer = FBMTrainer(**params)
    
    print(f"Training FBM (H={H}) policies...")
    trainer.train(n_paths=8192, epochs=3000)#6000
    
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=4096000)
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=16384 if H != 0.5 else 32768)#16384，32768
    
    print(f"\nFBM最优停止结果(H={H}):")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['upper']:.3f}, {ub_result['lower']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")

In [14]:
if __name__ == "__main__":
    
    print("========= Bermudan实验 =========")
    run_Bermudan_experiment()
    
    '''
    print("========= Callable MBRC实验 =========")
    run_mbrc_experiment()
    
    
    print("\n========= 分数布朗运动实验 =========")
    for H in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
             0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]:
        run_fbm_experiment(H)''
    '''

========= Bermudan实验 =========
BlackScholesGenerator steps: 9
开始训练...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

Training steps: 0it [00:00, ?it/s]

Training step 8:   0%|          | 0/1503 [00:00<?, ?it/s]

Training step 7:   0%|          | 0/1503 [00:00<?, ?it/s]

...

Lower bound: 18.150 (18.124, 18.176)
计算上界...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

预计算延续价值:   0%|          | 0/9 [00:00<?, ?it/s]

构建鞅过程:   0%|          | 0/9 [00:00<?, ?it/s]


Bermudan实验结果:
Lower bound: 18.150 (18.124, 18.176)
Upper bound: 20.984 (20.727, 21.242)
95% Confidence Interval: [18.124, 21.242]
Point Estimate: 19.567


In [15]:
def run_Bermudan_experiment():
    """Bermudan实验 对应论文表1/2 """
    params = {
        "d": 2,
        "steps": 9,
        "T": 3.0,
        "r": 0.05,
        "sigma": 0.2,
        "rho": 0.0,
        "div": 0.1
    }
    
    # 初始化生成器和训练器
    bs_gen = BlackScholesGenerator(**params)
    print("BlackScholesGenerator steps:", bs_gen.steps)
    trainer = BermudanMaxCallTrainer(bs_gen, hidden_dim=40+params["d"])
    
    # 训练（3000步，每批8192条路径）
    print("开始训练...")
    trainer.train(n_paths=4000, epochs=1500+params["d"],batch_size=2048)#8192,3000
    
    # 计算上下界
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)#4096000
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000) #1024，16384
    
    # 输出结果
    print(f"\nBermudan实验结果:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['lower']:.3f}, {ub_result['upper']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")
    
    del trainer
    del bs_gen
    del lb_result
    del ub_result

print("========= Bermudan实验 =========")
run_Bermudan_experiment()

========= Bermudan实验 =========
BlackScholesGenerator steps: 9
开始训练...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

Training steps: 0it [00:00, ?it/s]

Training step 8:   0%|          | 0/1502 [00:00<?, ?it/s]

Training step 7:   0%|          | 0/1502 [00:00<?, ?it/s]

...

Lower bound: 13.543 (13.521, 13.566)
计算上界...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

预计算延续价值:   0%|          | 0/9 [00:00<?, ?it/s]

构建鞅过程:   0%|          | 0/9 [00:00<?, ?it/s]


Bermudan实验结果:
Lower bound: 13.543 (13.521, 13.566)
Upper bound: 17.458 (16.995, 17.920)
95% Confidence Interval: [13.521, 17.920]
Point Estimate: 15.500


In [8]:
def run_Bermudan_experiment():
    """Bermudan实验 对应论文表1/2 """
    params = {
        "d": 5,
        "steps": 9,
        "T": 3.0,
        "r": 0.05,
        "sigma": 0.2,
        "rho": 0.0,
        "div": 0.1
    }
    
    # 初始化生成器和训练器
    bs_gen = BlackScholesGenerator(**params)
    print("BlackScholesGenerator steps:", bs_gen.steps)
    trainer = BermudanMaxCallTrainer(bs_gen, hidden_dim=40+params["d"])
    
    # 训练（3000步，每批8192条路径）
    print("开始训练...")
    trainer.train(n_paths=4000, epochs=1500+params["d"],batch_size=2048)#8192,3000
    
    # 计算上下界
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)#4096000
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000) #1024，16384
    
    # 输出结果
    print(f"\nBermudan实验结果:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['lower']:.3f}, {ub_result['upper']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")
    del trainer
    del bs_gen
    del lb_result
    del ub_result

print("========= Bermudan实验 =========")
run_Bermudan_experiment()

========= Bermudan实验 =========
BlackScholesGenerator steps: 9
开始训练...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

Training steps: 0it [00:00, ?it/s]

Training step 8:   0%|          | 0/1505 [00:00<?, ?it/s]

Training step 7:   0%|          | 0/1505 [00:00<?, ?it/s]

...

Lower bound: 25.416 (25.386, 25.445)
计算上界...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

预计算延续价值:   0%|          | 0/9 [00:00<?, ?it/s]

构建鞅过程:   0%|          | 0/9 [00:00<?, ?it/s]


Bermudan实验结果:
Lower bound: 25.416 (25.386, 25.445)
Upper bound: 34.863 (34.110, 35.616)
95% Confidence Interval: [25.386, 35.616]
Point Estimate: 30.139


In [9]:
def run_Bermudan_experiment():
    """Bermudan实验 对应论文表1/2 """
    params = {
        "d": 10,
        "steps": 9,
        "T": 3.0,
        "r": 0.05,
        "sigma": 0.2,
        "rho": 0.0,
        "div": 0.1
    }
    
    # 初始化生成器和训练器
    bs_gen = BlackScholesGenerator(**params)
    print("BlackScholesGenerator steps:", bs_gen.steps)
    trainer = BermudanMaxCallTrainer(bs_gen, hidden_dim=40+params["d"])
    
    # 训练（3000步，每批8192条路径）
    print("开始训练...")
    trainer.train(n_paths=4000, epochs=1500+params["d"],batch_size=2048)#8192,3000
    
    # 计算上下界
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)#4096000
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000) #1024，16384
    
    # 输出结果
    print(f"\nBermudan实验结果:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['lower']:.3f}, {ub_result['upper']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")
    del trainer
    del bs_gen
    del lb_result
    del ub_result

print("========= Bermudan实验 =========")
run_Bermudan_experiment()

========= Bermudan实验 =========
BlackScholesGenerator steps: 9
开始训练...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

Training steps: 0it [00:00, ?it/s]

Training step 8:   0%|          | 0/1510 [00:00<?, ?it/s]

Training step 7:   0%|          | 0/1510 [00:00<?, ?it/s]

...

Lower bound: 37.290 (37.257, 37.323)
计算上界...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

预计算延续价值:   0%|          | 0/9 [00:00<?, ?it/s]

构建鞅过程:   0%|          | 0/9 [00:00<?, ?it/s]


Bermudan实验结果:
Lower bound: 37.290 (37.257, 37.323)
Upper bound: 51.262 (50.394, 52.131)
95% Confidence Interval: [37.257, 52.131]
Point Estimate: 44.276


In [10]:
def run_Bermudan_experiment():
    """Bermudan实验 对应论文表1/2 """
    params = {
        "d": 1,
        "steps": 9,
        "T": 3.0,
        "r": 0.05,
        "sigma": 0.2,
        "rho": 0.0,
        "div": 0.1
    }
    
    # 初始化生成器和训练器
    bs_gen = BlackScholesGenerator(**params)
    print("BlackScholesGenerator steps:", bs_gen.steps)
    trainer = BermudanMaxCallTrainer(bs_gen, hidden_dim=40+params["d"])
    
    # 训练（3000步，每批8192条路径）
    print("开始训练...")
    trainer.train(n_paths=4000, epochs=1500+params["d"],batch_size=2048)#8192,3000
    
    # 计算上下界
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)#4096000
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000) #1024，16384
    
    # 输出结果
    print(f"\nBermudan实验结果:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['lower']:.3f}, {ub_result['upper']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")
    del trainer
    del bs_gen
    del lb_result
    del ub_result
    
print("========= Bermudan实验 =========")
run_Bermudan_experiment()

========= Bermudan实验 =========
BlackScholesGenerator steps: 9
开始训练...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

Training steps: 0it [00:00, ?it/s]

Training step 8:   0%|          | 0/1501 [00:00<?, ?it/s]

Training step 7:   0%|          | 0/1501 [00:00<?, ?it/s]

...

Lower bound: 7.873 (7.858, 7.888)
计算上界...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

预计算延续价值:   0%|          | 0/9 [00:00<?, ?it/s]

构建鞅过程:   0%|          | 0/9 [00:00<?, ?it/s]


Bermudan实验结果:
Lower bound: 7.873 (7.858, 7.888)
Upper bound: 9.764 (9.397, 10.132)
95% Confidence Interval: [7.858, 10.132]
Point Estimate: 8.819


In [ ]:
def run_Bermudan_experiment():
    """Bermudan实验 对应论文表1/2 """
    params = {
        "d": 20,
        "steps": 9,
        "T": 3.0,
        "r": 0.05,
        "sigma": 0.2,
        "rho": 0.0,
        "div": 0.1
    }
    
    # 初始化生成器和训练器
    bs_gen = BlackScholesGenerator(**params)
    print("BlackScholesGenerator steps:", bs_gen.steps)
    trainer = BermudanMaxCallTrainer(bs_gen, hidden_dim=40+params["d"])
    
    # 训练（3000步，每批8192条路径）
    print("开始训练...")
    trainer.train(n_paths=4000, epochs=1500+params["d"],batch_size=2048)#8192,3000
    
    # 计算上下界
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)#4096000
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000) #1024，16384
    # 输出结果
    print(f"\nBermudan实验结果:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['lower']:.3f}, {ub_result['upper']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")
    del trainer
    del bs_gen
    del lb_result
    del ub_result
    

print("========= Bermudan实验 =========")
run_Bermudan_experiment()

========= Bermudan实验 =========
BlackScholesGenerator steps: 9
开始训练...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

Training steps: 0it [00:00, ?it/s]

Training step 8:   0%|          | 0/1520 [00:00<?, ?it/s]

Training step 7:   0%|          | 0/1520 [00:00<?, ?it/s]

...

计算下界...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

Lower bound: 50.509 (50.475, 50.544)
计算上界...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

预计算延续价值:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
def run_Bermudan_experiment():
    """Bermudan实验 对应论文表1/2 """
    params = {
        "d": 30,
        "steps": 9,
        "T": 3.0,
        "r": 0.05,
        "sigma": 0.2,
        "rho": 0.0,
        "div": 0.1
    }
    
    # 初始化生成器和训练器
    bs_gen = BlackScholesGenerator(**params)
    print("BlackScholesGenerator steps:", bs_gen.steps)
    trainer = BermudanMaxCallTrainer(bs_gen, hidden_dim=40+params["d"])
    
    # 训练（3000步，每批8192条路径）
    print("开始训练...")
    trainer.train(n_paths=4000, epochs=1500+params["d"],batch_size=2048)#8192,3000
    
    # 计算上下界
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)#4096000
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000) #1024，16384
    
    # 输出结果
    print(f"\nBermudan实验结果:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['lower']:.3f}, {ub_result['upper']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")
    del trainer
    del bs_gen
    del lb_result
    del ub_result

print("========= Bermudan实验 =========")
run_Bermudan_experiment()

========= Bermudan实验 =========
BlackScholesGenerator steps: 9
开始训练...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

Training steps: 0it [00:00, ?it/s]

嵌套MC(t=8):   0%|          | 0/4000 [00:00<?, ?it/s]

Training step 8:   0%|          | 0/1530 [00:00<?, ?it/s]

...

Lower bound: 58.348 (58.189, 58.508)
计算上界...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

平均行权时间: 8.72 (步数), 对应时间: 2.91 年


Building Martingale:   0%|          | 0/9 [00:00<?, ?it/s]

时间步 1: 平均立即收益 = 0.0000, 平均继续价值 = 58.2838, ΔM均值 = 0.0000
时间步 2: 平均立即收益 = 23.2255, 平均继续价值 = 58.5525, ΔM均值 = 0.0000
时间步 3: 平均立即收益 = 32.9748, 平均继续价值 = 58.8152, ΔM均值 = 0.1532
时间步 4: 平均立即收益 = 39.1718, 平均继续价值 = 58.9022, ΔM均值 = 0.5291
时间步 5: 平均立即收益 = 44.5898, 平均继续价值 = 58.7401, ΔM均值 = 1.2707
时间步 6: 平均立即收益 = 48.6115, 平均继续价值 = 58.0519, ΔM均值 = 2.2519
时间步 7: 平均立即收益 = 52.1661, 平均继续价值 = 58.2606, ΔM均值 = 3.3211
时间步 8: 平均立即收益 = 53.7081, 平均继续价值 = 57.6177, ΔM均值 = 3.5844
时间步 9: 平均立即收益 = 55.7958, 平均继续价值 = 57.6323, ΔM均值 = 4.8944

Bermudan实验结果:
Lower bound: 58.348 (58.189, 58.508)
Upper bound: 65.153 (64.104, 66.201)
95% Confidence Interval: [58.189, 66.201]
Point Estimate: 61.751


In [ ]:
def run_Bermudan_experiment():
    """Bermudan实验 对应论文表1/2 """
    params = {
        "d": 50,
        "steps": 9,
        "T": 3.0,
        "r": 0.05,
        "sigma": 0.2,
        "rho": 0.0,
        "div": 0.1
    }
    
    # 初始化生成器和训练器
    bs_gen = BlackScholesGenerator(**params)
    print("BlackScholesGenerator steps:", bs_gen.steps)
    trainer = BermudanMaxCallTrainer(bs_gen, hidden_dim=40+params["d"])
    
    # 训练（3000步，每批8192条路径）
    print("开始训练...")
    trainer.train(n_paths=4000, epochs=1500+params["d"],batch_size=2048)#8192,3000
    
    # 计算上下界
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)#4096000
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000) #1024，16384
    
    # 输出结果
    print(f"\nBermudan实验结果:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['lower']:.3f}, {ub_result['upper']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")
    del trainer
    del bs_gen
    del lb_result
    del ub_result

print("========= Bermudan实验 =========")
run_Bermudan_experiment()

========= Bermudan实验 =========
BlackScholesGenerator steps: 9
开始训练...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

Training steps: 0it [00:00, ?it/s]

嵌套MC(t=8):   0%|          | 0/4000 [00:00<?, ?it/s]

Training step 8:   0%|          | 0/1550 [00:00<?, ?it/s]

...

Lower bound: 68.285 (68.123, 68.446)
计算上界...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

平均行权时间: 8.90 (步数), 对应时间: 2.97 年


Building Martingale:   0%|          | 0/9 [00:00<?, ?it/s]

时间步 1: 平均立即收益 = 0.0000, 平均继续价值 = 68.3374, ΔM均值 = 0.0000
时间步 2: 平均立即收益 = 26.5566, 平均继续价值 = 68.4754, ΔM均值 = 0.0000
时间步 3: 平均立即收益 = 37.0511, 平均继续价值 = 68.4549, ΔM均值 = 0.0686
时间步 4: 平均立即收益 = 44.6846, 平均继续价值 = 68.3849, ΔM均值 = 0.4311
时间步 5: 平均立即收益 = 50.7204, 平均继续价值 = 68.1646, ΔM均值 = 0.9531
时间步 6: 平均立即收益 = 55.7072, 平均继续价值 = 67.9812, ΔM均值 = 1.9707
时间步 7: 平均立即收益 = 60.1603, 平均继续价值 = 68.2940, ΔM均值 = 2.9694
时间步 8: 平均立即收益 = 63.2544, 平均继续价值 = 68.1831, ΔM均值 = 3.7916
时间步 9: 平均立即收益 = 66.4265, 平均继续价值 = 68.4692, ΔM均值 = 4.8291

Bermudan实验结果:
Lower bound: 68.285 (68.123, 68.446)
Upper bound: 74.130 (73.063, 75.196)
95% Confidence Interval: [68.123, 75.196]
Point Estimate: 71.207


In [ ]:
def run_Bermudan_experiment():
    """Bermudan实验 对应论文表1/2 """
    params = {
        "d": 100,
        "steps": 9,
        "T": 3.0,
        "r": 0.05,
        "sigma": 0.2,
        "rho": 0.0,
        "div": 0.1
    }
    
    # 初始化生成器和训练器
    bs_gen = BlackScholesGenerator(**params)
    print("BlackScholesGenerator steps:", bs_gen.steps)
    trainer = BermudanMaxCallTrainer(bs_gen, hidden_dim=40+params["d"])
    
    # 训练（3000步，每批8192条路径）
    print("开始训练...")
    trainer.train(n_paths=4000, epochs=1500+params["d"],batch_size=2048)#8192,3000
    
    # 计算上下界
    print("计算下界...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)#4096000
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print("计算上界...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000) #1024，16384
    
    # 输出结果
    print(f"\nBermudan实验结果:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['lower']:.3f}, {ub_result['upper']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")
    del trainer
    del bs_gen
    del lb_result
    del ub_result

print("========= Bermudan实验 =========")
run_Bermudan_experiment()

========= Bermudan实验 =========
BlackScholesGenerator steps: 9
开始训练...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

Training steps: 0it [00:00, ?it/s]

嵌套MC(t=8):   0%|          | 0/4000 [00:00<?, ?it/s]

Training step 8:   0%|          | 0/1600 [00:00<?, ?it/s]

...

Lower bound: 82.086 (81.923, 82.250)
计算上界...


生成Black-Scholes路径:   0%|          | 0/9 [00:00<?, ?it/s]

平均行权时间: 8.99 (步数), 对应时间: 3.00 年


Building Martingale:   0%|          | 0/9 [00:00<?, ?it/s]

时间步 1: 平均立即收益 = 0.0000, 平均继续价值 = 82.2013, ΔM均值 = 0.0000
时间步 2: 平均立即收益 = 30.0259, 平均继续价值 = 82.1543, ΔM均值 = 0.0000
时间步 3: 平均立即收益 = 42.6726, 平均继续价值 = 82.2405, ΔM均值 = 0.0140
时间步 4: 平均立即收益 = 51.4013, 平均继续价值 = 82.1401, ΔM均值 = 0.1305
时间步 5: 平均立即收益 = 58.6397, 平均继续价值 = 81.9811, ΔM均值 = 0.5767
时间步 6: 平均立即收益 = 65.4755, 平均继续价值 = 82.2805, ΔM均值 = 1.4418
时间步 7: 平均立即收益 = 70.6476, 平均继续价值 = 82.4111, ΔM均值 = 2.2744
时间步 8: 平均立即收益 = 74.6769, 平均继续价值 = 81.9458, ΔM均值 = 3.0325
时间步 9: 平均立即收益 = 78.4296, 平均继续价值 = 81.7484, ΔM均值 = 4.5627

Bermudan实验结果:
Lower bound: 82.086 (81.923, 82.250)
Upper bound: 88.284 (87.131, 89.436)
95% Confidence Interval: [81.923, 89.436]
Point Estimate: 85.185
